In [1]:
pip install langchain openai pdfplumber python-dotenv


  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached pdfplumber-0.11.7-py3-none-any.whl.metadata (42 kB)
  Using cached langchain_core-0.3.78-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached langsmith-0.4.32-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.11.10-py3-none-any.whl.metadata (68 kB)
  Using cached sqlalchemy-2.0.43-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pdfminer_six-20250506-py3-none-any.whl.metadata (4.2 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached pypdfium2-4.30.0-py3


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install langchain_community

  Using cached numpy-2.3.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 76.3/76.3 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 10.9 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.5 MB 11.4 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 8.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.5 MB 10.0 MB/s eta 0:00:01
   ----------------------------------- ---- 2.2/2.5 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------  2.5/2.5 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 9.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/450.3 kB ? eta -:--:--
   --------------------------------------  440.3/450.3 kB 13.9 MB/s eta 0:00:01
   -------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import json
import pdfplumber
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from openai import OpenAI
import pdfplumber
import json

In [11]:
# ---- your OpenRouter setup ----
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

In [12]:
# ---- helper: extract text from pdf ----
def extract_text_from_pdf(file_path: str) -> str:
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text.strip()

In [ ]:
# ---- main extraction function ----
def extract_transactions(file_path: str, user_id: str):
    statement_text = extract_text_from_pdf(file_path)[:15000]  # keep under token limit

    prompt = f"""
Extract all transactions from the text below and return them as a valid JSON array.
Each transaction must include:
- id (string)
- userId (string)
- date (date)
- description (string)
- amount (number)
- type (string: "Credit" or "Debit")
- category (string)
- balance (number)

Text:
{statement_text}
"""

    completion = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "https://your-site-url.com",  # optional
            "X-Title": "Transaction Extractor",            # optional
        },
        model="gpt-4o-mini",  # or any OpenRouter-supported model, e.g., qwen/qwen3-235b-a22b-2507
        messages=[{"role": "user", "content": prompt}],
    )

    response_text = completion.choices[0].message.content

    # try parsing response as JSON
    try:
        data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Model output wasn't valid JSON:\n", response_text)
        data = []

    # ensure required fields
    for i, t in enumerate(data):
        t["id"] = t.get("id") or f"txn-{i+1}"
        t["userId"] = t.get("userId") or user_id

    return data

In [14]:
if __name__ == "__main__":
    transactions = extract_transactions(r"C:\Users\HP\Documents\Suvit\hdfc-demo.pdf", "user123")
    print(json.dumps(transactions, indent=2))


⚠️ Model output wasn't valid JSON:
 Here are the transactions extracted from the provided text formatted as a valid JSON array:

```json
[
    {
        "id": "001",
        "userId": "3327106",
        "date": "2025-03-01",
        "description": "NEFTCR-CITI0100000-INDIAIDEAS.COMLIMIT",
        "amount": 12888.00,
        "type": "Credit",
        "category": "Transfer",
        "balance": 1386900.83
    },
    {
        "id": "002",
        "userId": "3327106",
        "date": "2025-03-01",
        "description": "UPI-MRSZULEKHAHAMIDSA-ZULEKHA.PEERZAD",
        "amount": 806.00,
        "type": "Credit",
        "category": "UPI",
        "balance": 1387706.83
    },
    {
        "id": "003",
        "userId": "3327106",
        "date": "2025-03-01",
        "description": "CASHDEPOSITBY-SELF-KOREGAONPARK",
        "amount": 3400.00,
        "type": "Credit",
        "category": "Deposit",
        "balance": 1391106.83
    },
    {
        "id": "004",
        "userId": "3327106",
